In [1]:
import pandas as pd

df = pd.read_csv("student_writing.csv")

# Group tokens by text_id to form full text
texts = (
    df.sort_values("position")
      .groupby("text_id")
      .agg({
          "token": list,
          "target": list
      })
      .reset_index()
)

texts.head()


,text_id,token,target
0,S1,"[He, go, to, university, every, day, .]","[0, 1, 0, 0, 0, 0, 0]"
1,S10,"[This, are, the, students, .]","[1, 0, 0, 0, 0]"
2,S100,"[This, are, the, students, .]","[1, 0, 0, 0, 0]"
3,S101,"[I, has, two, brothers, .]","[0, 1, 0, 0, 0]"
4,S102,"[He, drive, very, fastly, .]","[0, 1, 0, 1, 0]"


In [2]:
def compute_error_rate(targets):
    return sum(targets) / len(targets)

texts["error_rate"] = texts["target"].apply(compute_error_rate)
#After this every sample will have token_list and the error_rate

In [4]:
#Create the actual text string
texts["text"] = texts["token"].apply(lambda x: " ".join(x))


In [5]:
def assign_label(error_rate):
    if error_rate >= 0.15:
        return 1  # dyslexic_like
    elif error_rate <= 0.05:
        return 0  # normal
    else:
        return None  # discard

texts["label"] = texts["error_rate"].apply(assign_label)

clean_texts = texts.dropna(subset=["label"])


In [6]:
import os

base_path = "data/language/raw"
os.makedirs(f"{base_path}/dyslexic_like", exist_ok=True)
os.makedirs(f"{base_path}/normal", exist_ok=True)

for idx, row in clean_texts.iterrows():
    label_folder = "dyslexic_like" if row["label"] == 1 else "normal"
    file_path = f"{base_path}/{label_folder}/sample_{row['text_id']}.txt"
    
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(row["text"])



In [7]:
!pip install pdfplumber


   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   - -------------------------------------- 0.3/5.6 MB ? eta -:--:--
   --- ------------------------------------ 0.5/5.6 MB 2.1 MB/s eta 0:00:03
   ------- -------------------------------- 1.0/5.6 MB 1.9 MB/s eta 0:00:03
   --------- ------------------------------ 1.3/5.6 MB 1.9 MB/s eta 0:00:03
   ----------- ---------------------------- 1.6/5.6 MB 1.6 MB/s eta 0:00:03
   -------------- ------------------------- 2.1/5.6 MB 1.7 MB/s eta 0:00:03
   ---------------- ----------------------- 2.4/5.6 MB 1.7 MB/s eta 0:00:02
   -------------------- ------------------- 2.9/5.6 MB 1.8 MB/s eta 0:00:02
   ---------------------- ----------------- 3.1/5.6 MB 1.7 MB/s eta 0:00:02
   ------------------------ --------------- 3.4/5.6 MB 1.7 MB/s eta 0:00:02
   -------------------------- ------------- 3.7/5.6 MB 1.6 MB/s eta 0:00:02
   --------------------------- ------------ 3.9/5.6 MB 1.6 MB/s eta 0:00:02
   -----------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tableauhyperapi 0.0.24081 requires cffi!=1.14.3,<2,>=1.12.2, but you have cffi 2.0.0 which is incompatible.
